**The procedure goes on three steps: trim the video to video clips, get the classification results from the video clips and output the classification results in json files**

## Trim video to video clips and rename them

In [ ]:
# install the package moviepy 
!pip install moviepy

In [ ]:
# get access from the google drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /

In [ ]:
# Import everything needed to edit video clips
from moviepy.editor import *

In [ ]:
#set the video's original directory and the directory for the video clips
import os
import numpy as np
import math

base_dir='content/gdrive/My Drive/CSCE636_Deep_Learning/Action Detection_1'   #video's original directory
video_list=os.listdir(base_dir)
dst_dir='content/gdrive/My Drive/CSCE636_Deep_Learning/Action_Detect_Clips/Test1/Test1'  #two recursive test folders are needed because the videogenerator I used requires them
os.mkdir(dst_dir)

In [ ]:
# clip video to video clips every 6 seconds  
i=0
interval=6
for video in video_list:
  video_src_dir=os.path.join(base_dir,video)
  myclip = VideoFileClip(video_src_dir,audio=False)
  i +=1
  for time in np.arange(myclip.duration // interval):
    video_dst=video.split(".")[0]+"+"+str(time*interval)+"+"+str(time*interval+2)+'.mp4'    #rename the video clips as videoId+start time+end time
    video_dst_dir=os.path.join(dst_dir,video_dst)
    output_clip=myclip.subclip(t_start=time*interval, t_end=time*interval+2)
    output_clip.write_videofile(video_dst_dir) 
    print('Clipping:',i,' Duration:',myclip.duration)
    print('Clipping time:',time*interval,time*interval+2)


## Predict on test video

In [ ]:
#install the package for the video generators
!pip install keras-video-generators

In [ ]:
#get access from google drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /

In [ ]:
#set the directory of training data and test data
train_dir='/content/gdrive/My Drive/CSCE636_Deep_Learning/Model/Custom_Dataset/Custom_Dataset4/{classname}/*'

testset_name='Test1'
test_dir='content/gdrive/My Drive/CSCE636_Deep_Learning/Action_Detect_Clips/'+testset_name+'/{classname}/*'

In [ ]:
#import the model I trained
import keras 
model_name='model_custom4_1'
model = keras.models.load_model('content/gdrive/My Drive/CSCE636_Deep_Learning/Model/chkp/'+model_name+'.h5')

In [ ]:
#keep the global parameters as the training model
size = (224, 224)
channels = 3
N_frames = 8
Batch_size = 8
classes=2
train_validate_split=0.3

In [ ]:
# from training data generator to get labels
from keras_video import VideoFrameGenerator
from keras_preprocessing.image import ImageDataGenerator

data_aug = ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

train_gen = VideoFrameGenerator( 
    glob_pattern=train_dir,
    nb_frames=N_frames,
    split_val=train_validate_split, 
    shuffle=True,
    batch_size=Batch_size,
    target_shape=size,
    nb_channel=channels,
    transformation=data_aug,
    use_frame_cache=False)

In [ ]:
#set test data generator for testing
test_gen = VideoFrameGenerator(
    glob_pattern=test_dir,
    nb_frames=N_frames, 
    batch_size=1,
    target_shape=size,
    nb_channel=channels,
    shuffle=False,
    use_frame_cache=False)

Total data: 1 classes for 3284 files for train


In [ ]:
#show the number of video clips for testing
len(test_gen.files)

In [ ]:
#prediction using test data
predict = model.predict_generator(test_gen)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
#show the prediction results
preds_cls_idx = predict.argmax(axis=-1)
print(preds_cls_idx)

In [ ]:
#show the prediction files' dimension
predict.shape

In [ ]:
#show the classes order 
train_gen.classes

In [ ]:
#make a dictionary for the labels and classes generated by model
class_names = sorted(train_gen.classes) # Sorting them
name_id_map = dict(zip(class_names, range(len(class_names))))

In [ ]:
#show the dictionary of lables
name_id_map

In [ ]:
#show the testing data
from pandas import DataFrame
FileName=DataFrame(test_gen.files, columns=['FileName'])
FileName

In [ ]:
#show the prediction results
ClassName=DataFrame(preds_cls_idx, columns=['ClassName'])
ClassName.columns

In [ ]:
#combine the testing data with their prediciton results
import pandas as pd

output=pd.concat([FileName, ClassName], axis=1)
output

In [ ]:
#keep the target action for output
output_json=output[output.ClassName==0]
output_json.FileName

## Write json file

In [ ]:
# get access from google drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /

In [ ]:
# set the directory for the output json file
import os
json_dir='/content/gdrive/My Drive/CSCE636_Deep_Learning/Output/Submission2'
try: 
  os.mkdir(json_dir)
except:
  print("Folder exists")

Folder exists


In [ ]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
#show the number of videoclips which contain the target action (Clapping_hands)
len(output_json)

In [ ]:
#show the the videoId, start time and end time of the output videoclips
for address in output_json['FileName']:
  video_name=address.split("/")[-1]
  print(video_name.split("+")[0],video_name.split("+")[1],video_name.split("+")[2].split(".")[0])

In [ ]:
#output the json file
## the information of every 300 videoclips are combined in a json file
df=[]
nick_name="CSCE636Spring2021-RZ241-2"
label="Clapping-hands"

max_number=300
file_num=0
name_base="ResultOfDetection_RZ241_submission2_"+model_name+"_"+testset_name
video_count=0

for address in output_json['FileName']:
  video_count +=1
  video_name=address.split("/")[-1]
  videoid=video_name.split("+")[0]
  video_start=video_name.split("+")[1]
  video_end=video_name.split("+")[2].split(".")[0]
  df_tmp={"videoId":videoid,"type":"segment","startTime":float(video_start),"endTime":float(video_end),"observer":nick_name,"isHuman":False,
       "confirmedBySomeone": False,"rejectedBySomeone": False,"observation":{"label": label,"labelConfidence": 0.85}}
  df.append(df_tmp)
  
  if (video_count % max_number==0):
    file_num +=1
    filename=name_base+str(file_num)+".json"
    with open(os.path.join(json_dir,filename), 'w',encoding='utf-8') as f:
      json.dump(df, f,ensure_ascii=False, indent=4)
    df=[]

if (video_count % max_number>0):
  file_num +=1
  filename=name_base+str(file_num)+".json"
  with open(os.path.join(json_dir,filename), 'w',encoding='utf-8') as f:
    json.dump(df, f,ensure_ascii=False, indent=4)